
# Time Series CV

Time series cross validation is deserves special treatment. This is due in part because of autocorrelction factors. `tablespoon` offerts a `TimeSeriesInitialSplit` class with the `split` method. It was designed to fit in naturally with what some users may be used to from `sklearn`. 

Some may wonder why `sklearn` is not sufficient. It is possible to use the methods in `sklearn` to make a work-around solution. This method is more natural as it allows users to explicitly define their intial time period, increment size, and gap. 


## Rolling Origin Cross Validation

![image](https://raw.githubusercontent.com/alexhallam/tablespoon/main/examples/cv1-1.svg)

Assume the blue dots represents dates to train on and the red dots are for testing.

In this picture the initial size of the blue dots are much larger than the size of the testing dots. (6 blue dots vs 1 red). With each iteration train size increases by 1 blue dot.

`TimeSeriesInitialSplit` allows this kind of flexibility. A user may set a larger initial training period, which is not bound by the size that the training set is increases by on each iteration. It decouples the initial size from the increment size.

# Example 1

This example simple shows the functionality `TimeSeriesInitialSplit`. Note the initial time period is set to 7 and increments by 7 as well. There is no gap between the training periods and the test periods. 

In [1]:
import numpy as np
from tablespoon.model_selection import TimeSeriesInitialSplit

X = np.arange(0, 28)
tscv = TimeSeriesInitialSplit(initial=7, increment_size=7, gap=0)
fold_counter = 0
for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index,"TEST:", test_index)
    fold_counter += 1
    print(f"fold number: {fold_counter}")

ModuleNotFoundError: No module named 'tablespoon.model_selection'

# Example 2

This is closer to a real world example.

In [ ]:
import numpy as np
from tablespoon.model_selection import TimeSeriesInitialSplit
from tablespoon.data import WALMART_TX
from tablespoon.forecasters import Snaive, Mean, Naive
import pandas as pd

X = WALMART_TX.assign(ds = lambda df: pd.to_datetime(df.ds))
tscv = TimeSeriesInitialSplit(initial= 365, increment_size=7, gap=2)
fold_counter = 0
vec_mean = np.asarray([])
vec_naive = np.asarray([])
vec_snaive = np.asarray([])

def crps(scalar_y, vec_of_forecast):
    x = np.sort(vec_of_forecast)
    m = len(vec_of_forecast)
    return (2 / m) * np.mean((x - scalar_y) * (m * np.where(scalar_y < x, 1, 0) - np.arange(start=0, stop=m, step=1) + 1 / 2))
for train_index, test_index in tscv.split(X):
    train, test = X.iloc[train_index], X.iloc[test_index]
    h = test.shape[0]
    # make some forecasts from different models
    df_n = (Naive().predict(df_historical = train, horizon=h, frequency="D", lag = 1, uncertainty_samples = 500).assign(model = 'naive'))    
    df_sn = (Snaive().predict(df_historical = train, horizon=h, frequency="D", lag = 7, uncertainty_samples = 500).assign(model = 'snaive'))
    df_m = (Mean().predict(df_historical = train, horizon=h, frequency="D", uncertainty_samples = 500).assign(model = 'mean'))
    # merge each test set to each model's forecast
    df_forecast_actual_n = test.merge(df_n, how='left', on = 'ds')    
    df_forecast_actual_sn = test.merge(df_sn, how='left', on = 'ds')
    df_forecast_actual_m = test.merge(df_m, how='left', on = 'ds')
    # put all the forecast models together
    df_forecasts = pd.concat([df_forecast_actual_n, df_forecast_actual_sn, df_forecast_actual_m], axis=0)
    # calculate mean crps by model type
    df_crps = (df_forecasts
            .groupby(by = ['model'], as_index=False)
            .apply(lambda df: crps(df["y"].iat[0], df["y_sim"]))
            .assign(metric="crps")
            .rename(columns={None: "value"}))
    score_mean = df_crps.value # mean, naive, snaive
    vec_mean = np.concatenate((vec_mean, score_mean[0].flatten()), axis = 0)
    vec_naive = np.concatenate((vec_naive, score_mean[1].flatten()), axis = 0)
    vec_snaive = np.concatenate((vec_snaive, score_mean[2].flatten()), axis = 0)

Every forecast method in `tablespoon` is a reference/benchmark method. Find which of the forecast methods perform the best on average. That will be used as the reference.

To calculate the skill score the following formulua is used.

$$
\frac{\text{CRPS}_{\text{Naïve}} - \text{CRPS}_{\text{Alternative}}}{\text{CRPS}_{\text{Naïve}}}
$$

With some simplification this is the same as the following

$$
1 - \frac{\text{CRPS}_{\text{Alternative}}}{\text{CRPS}_{\text{Naïve}}}
$$

In [ ]:
avg_mean_score = np.mean(vec_mean)
avg_naive_score = np.mean(vec_naive)
avg_snaive_score = np.mean(vec_snaive)
print(f'avg_mean_score {np.round(avg_mean_score, 3)}')
print(f'avg_naive_score {np.round(avg_naive_score, 3)}')
print(f'avg_snaive_score {np.round(avg_snaive_score, 3)}')

avg_mean_score 557.113
avg_naive_score 1064.276
avg_snaive_score 585.291


Of all the benchmark method the `Mean` is the best performer. This will be used as a the reference for the skill score calculation.

In [ ]:
def skill_score(best_naive_method, alternative):
  return 1 - (alternative / best_naive_method)

In [ ]:
mean_skill = skill_score(avg_mean_score, avg_mean_score)
naive_skill = skill_score(avg_mean_score, avg_naive_score)
snaive_skill = skill_score(avg_mean_score, avg_snaive_score)
print(f"mean_skill {np.round(mean_skill, 3)}")
print(f"naive_skill {np.round(naive_skill, 3)}")
print(f"snaive_skill {np.round(snaive_skill, 3)}")

mean_skill 0.0
naive_skill -0.91
snaive_skill -0.051


## Notes on Skill Scores

1. If the alternative forecast error is equal to the forecast error of the reference then the skill score will be `0`.

2. If the alternative forecast error is perfect, score_error = 0, then then the skill score will be `1`.

3. If the alternative forecast error is worse than the reference then the skill score will result in a value `< 0`.

In summary, skill scores have the range `[-inf, 1]`